<a href="https://colab.research.google.com/github/Ni-cole17/Diagnostic-ECG-Database/blob/main/Trabalho_Tecnicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importações

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wfdb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 161 kB 4.0 MB/s 


In [13]:
import wfdb
import pandas as pd
import numpy as np
import re
import csv
import os

In [4]:
!git clone https://github.com/MIT-LCP/wfdb-python.git

Cloning into 'wfdb-python'...
remote: Enumerating objects: 5699, done.
remote: Counting objects: 100% (610/610), done.
remote: Compressing objects: 100% (299/299), done.
remote: Total 5699 (delta 406), reused 440 (delta 303), pack-reused 5089
Receiving objects: 100% (5699/5699), 167.03 MiB | 15.39 MiB/s, done.
Resolving deltas: 100% (3752/3752), done.
Checking out files: 100% (276/276), done.


In [5]:
%cd wfdb-python

/content/wfdb-python


## Visualização (Pedro)


## Inicio (Nathalia)

In [ ]:
# Função que lê sinal e paciente 
def read(a,b): 
  signals, fields = wfdb.rdsamp(a, channels=[*range(15)], sampfrom=100, sampto=15000, pn_dir='ptbdb/'+ b + '/')
  return signals, fields

#Ex.: x = read('s0010_re','patient001')

In [ ]:
#Função que converte a formata a lista "comment para poder passar ela para dicionário"
def FormataLista(a):
  ListaFormatada = []
  for i in range(len(a)):
    ListaFormatada.append(re.sub(r': +',"//",a[i].strip()).split("//"))
  return ListaFormatada

In [ ]:
#Função que converte lista para dicionário
def Convert(lst):
    res_dct = {lst[i]: lst[i + 1] for i in range(0, len(lst), 2)}
    return res_dct

In [ ]:
#Função que organiza documento de acordo com a base:
def OrganizeFile(a,b):

  Fields, Signals = read(a,b)
  ListaFormatada = FormataLista(Fields['comments'])

  paciente = {}
  hemodynamics = {}
  therapy = {}
  HeartAttack = {}
  Signal = {}

  #Organizando Pacients
  list1 = [0, 1, 7, 8]
  for i in list1:
    paciente.update(Convert(ListaFormatada[i]))
  
  #Organizando hemodynamics
  list2 = [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]
  for i in list2:
    hemodynamics.update(Convert(ListaFormatada[i]))

  #Organizando Therapy
  list3 = [38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
  for i in list3:
    therapy.update(Convert(ListaFormatada[i]))

  #Organizando HeartAttack
  list4 = [5, 6, 9, 10, 11, 12]
  for i in list4:
    HeartAttack.update(Convert(ListaFormatada[i]))

  #Organizando Signals
  #Signal.update(Signals)

  #list0 = [2]
  #for i in list0:
  #  Signal.update(Convert(ListaFormatada[i]))

 # del Fields['comments']
  #Signal.update(Fields)


  # Retornar o dicionario e o sinal
  return paciente, hemodynamics, therapy, HeartAttack

## Classes objeto (Nicole)

In [363]:
class Object:
  def getdados(self):
    pass
  def savedados(self):
    pass

In [364]:
class Registro:
  def __init__(self,dictionary,signal,new_patient,id = 'None'):
    self.dictionary = dictionary
    self.signal = signal
    self.new_patient = new_patient
    self.idP = id

  def save_data(self):
    if self.new_patient == True:
      patient = Patient(self.dictionary)
      patient.savedados()
      self.idP = patient.idP
      infarction = Infarction(self.dictionary,self.idP)
      infarction.savedados()
      hemodinamics = Hemodinamics(self.dictionary,self.idP)
      hemodinamics.savedados()
      therapy = Therapy(self.dictionary,self.idP)
      therapy.savedados()
      self.save_signal()
    else:
      self.idP = self.idP
      self.save_signal()
    return self.idP

  def save_signal(self):
    signal = Signal(self.dictionary,self.idP,self.signal,self.new_patient)
    signal.savedados()

In [365]:
class Patient(Object):
  def __init__(self, patient_dict):
    self.__patient_dict = patient_dict
    self.__csv = 'patient.csv'
    self.__keys = ['age','sex','Smoker','Reason for admission','Additional diagnoses']

  def __generateid(self):
    if os.path.isfile(self.__csv) == True:
      idP = sorted(pd.read_csv(self.__csv)['idP'].unique())[-1] + 1
    else:
      idP = 0
    return idP

  def __getdados(self):
    patient = {}
    self.idP = self.__generateid()
    patient['idP'] = self.idP
    for i in self.__keys:
      patient[i] = self.__patient_dict[i]
    return patient
  

  def savedados(self):
    patient = self.__getdados()
    k = Salva(patient,self.__csv)
    k.salva()


In [366]:
class Infarction(Object):
  def __init__(self, patient_dict,idP):
    self.__infarction_dict = patient_dict
    self.__csv = 'infarction.csv'
    self.__keys = ['Acute infarction (localization)','Former infarction (localization)','Number of coronary vessels involved','Infarction date (acute)',
                   'Previous infarction (1) date','Previous infarction (1) date']
    self.__idP = idP

  def __getdados(self):
    infarction = {}
    infarction['idP'] = self.__idP
    for i in self.__keys:
      infarction[i] = self.__infarction_dict[i]
    return infarction


  def savedados(self):
    infarction = self.__getdados()
    k = Salva(infarction,self.__csv)
    k.salva()


In [367]:
class Hemodinamics(Object):
  def __init__(self, patient_dict,idP):
    self.__hemodinamics_dict = patient_dict
    self.__csv = 'hemodinamics.csv'
    self.__keys = ['Catheterization date', 'Ventriculography','Chest X-ray','Peripheral blood Pressure (syst/diast)','Pulmonary artery pressure (at rest) (syst/diast)',
                   'Pulmonary artery pressure (at rest) (mean)','Pulmonary capillary wedge pressure (at rest)','Cardiac output (at rest)','Cardiac index (at rest)',
                   'Stroke volume index (at rest)','Pulmonary artery pressure (laod) (syst/diast)','Pulmonary artery pressure (laod) (mean)','Pulmonary capillary wedge pressure (load)',
                   'Cardiac output (load)','Cardiac index (load)','Stroke volume index (load)','Aorta (at rest) (syst/diast)','Aorta (at rest) mean','Left ventricular enddiastolic pressure',
                   'Left coronary artery stenoses (RIVA)','Left coronary artery stenoses (RCX)','Right coronary artery stenoses (RCA)','Echocardiography']
    self.__idP = idP

  def __getdados(self):
    hemodinamics = {}
    hemodinamics['idP'] = self.__idP
    for i in self.__keys:
      hemodinamics[i] = self.__hemodinamics_dict[i]
    return hemodinamics

  def savedados(self):
    hemodinamics = self.__getdados()
    k = Salva(hemodinamics,self.__csv)
    k.salva()

In [368]:
class Therapy(Object):
  def __init__(self, patient_dict,idP):
    self.__therapy_dict = patient_dict
    self.__csv = 'therapy.csv'
    self.__keys = ['Admission date','Medication pre admission','Start lysis therapy (hh.mm)','Lytic agent','Dosage (lytic agent)','Additional medication',
                   'In hospital medication','Medication after discharge']
    self.__idP = idP

  def __getdados(self):
    therapy = {}
    therapy['idP'] = self.__idP
    for i in self.__keys:
      therapy[i] = self.__therapy_dict[i]
    return therapy

  def savedados(self):
    therapy = self.__getdados()
    k = Salva(therapy,self.__csv)
    k.salva()

In [369]:
class Signal(Object):
  def __init__(self,patient_dict,idP,signal,new_patient):
    self.__signal_dict = patient_dict
    self.__csv = 'signal.csv'
    self.__idP = idP
    self.__signal = signal
    self.__new_patient = new_patient
    self.__keys = ['fs','sig_len','n_sig','base_date','base_time','units','sig_name']

  def __generateid(self):
    csv_dir = os.path.join('/content',self.__csv)
    if os.path.isfile(csv_dir) == True:
      if self.__new_patient == False:
        df = pd.read_csv(csv_dir).query(f"idP == {self.__idP}")
        idS = sorted(df['idS'].unique())[-1] + 1
      else:
        idS = 0
    else:
      idS = 0
    return idS
  
  def __getdados(self):
    signal = {}
    self.__idS = self.__generateid()
    signal['idP'] = self.__idP
    signal['idS'] = self.__idS
    for i in self.__keys:
      signal[i] = self.__signal_dict[i]  
    signal['Signal'] = self.__signal

    return signal

  def savedados(self):
    signal = self.__getdados()
    k = Salva(signal,self.__csv)
    k.salva()


In [370]:
class Salva:
  def __init__(self,some_dict,csv):
    self.csv = csv
    self.some_dict = some_dict
  
  def read_csv(self):
    df = pd.read_csv(self.csv)
    return df

  def salva(self): # Falta adicionar o idPaciente
    dir = os.path.join('/content', self.csv)
    if os.path.isfile(dir) == False:
      header = []
      row = []
      with open(dir, 'a') as csv_file:
        writer = csv.writer(csv_file)
        for key, value in self.some_dict.items():
          header.append(key)
          row.append(value) 
        writer.writerow(header)
        writer.writerow(row)
    else:
      with open(dir, 'a') as csv_file:  
        writer = csv.writer(csv_file)
        row = []
        for key, value in self.some_dict.items():
          row.append(value)
        writer.writerow(row)

## Tests

In [359]:
my_json = dict(pd.read_json('/content/drive/MyDrive/Projeto-Tecnicas/patient_heas.json',typ='series'))

In [360]:
my_json

{'patient253': {'id': 0, 'heas': ['s0488_re.hea']},
 'patient201': {'id': 1, 'heas': ['s0420_re.hea', 's0423_re.hea']},
 'patient087': {'id': 2,
  'heas': ['s0330lre.hea', 's0321lre.hea', 's0326lre.hea']},
 'patient254': {'id': 3, 'heas': ['s0489_re.hea']},
 'patient065': {'id': 4,
  'heas': ['s0282lre.hea', 's0229lre.hea', 's0226lre.hea', 's0221lre.hea']},
 'patient196': {'id': 5, 'heas': ['s0002_re.hea']},
 'patient059': {'id': 6, 'heas': ['s0208lre.hea']},
 'patient283': {'id': 7, 'heas': ['s0542_re.hea']},
 'patient185': {'id': 8, 'heas': ['s0336lre.hea']},
 'patient121': {'id': 9, 'heas': ['s0311lre.hea']},
 'patient277': {'id': 10, 'heas': ['s0527_re.hea']},
 'patient034': {'id': 11,
  'heas': ['s0158lre.hea', 's0109lre.hea', 's0118lre.hea', 's0123lre.hea']},
 'patient007': {'id': 12,
  'heas': ['s0078lre.hea', 's0038lre.hea', 's0026lre.hea', 's0029lre.hea']},
 'patient017': {'id': 13,
  'heas': ['s0075lre.hea', 's0053lre.hea', 's0063lre.hea', 's0055lre.hea']},
 'patient019': {'i

In [274]:
def read(a,b): 
  signals, fields = wfdb.rdsamp(a, channels=[*range(15)], sampfrom=100, sampto=15000, pn_dir='ptbdb/'+ b + '/')
  return signals, fields

In [275]:
def FormataLista(a):
  ListaFormatada = []
  for i in range(len(a)):
    ListaFormatada.append(re.sub(r': +',"//",a[i].strip()).split("//"))
  ListaFormatada.remove(['Diagnose:'])
  ListaFormatada.remove(['Hemodynamics:'])
  ListaFormatada.remove(['Therapy:'])
  return ListaFormatada

In [276]:
#Função que converte lista para dicionário
def Convert(listt):
  res_dct = {}
  for lst in listt:
    res_dct[lst[0]] =  lst[1] 
  return res_dct

In [277]:
def merge_dict(dict1,dict2):
  dict1.pop('comments', None)
  for k,v in dict2.items():
    dict1[k] = v
  return dict1

In [361]:
count = 0
for k,v in my_json.items():
  new_patient = True
  count +=1
  id ='None'
  for value in v['heas']:
    signal, fields = read(value.replace('.hea',''),k)
    b = FormataLista(fields['comments'])
    c = Convert(b)
    dictionary = merge_dict(fields,c)
    register = Registro(dictionary,signal,new_patient,id=id)
    id = register.save_data()
    #print(dictionary)
    new_patient = False
  if count == 10:
    break

id_patient:  0
{'fs': 1000, 'sig_len': 14900, 'n_sig': 15, 'base_date': None, 'base_time': None, 'units': ['mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV'], 'sig_name': ['i', 'ii', 'iii', 'avr', 'avl', 'avf', 'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'vx', 'vy', 'vz'], 'age': '62', 'sex': 'male', 'ECG date': '18/02/1997', 'Reason for admission': 'Cardiomyopathy', 'Acute infarction (localization)': 'no', 'Former infarction (localization)': 'no', 'Additional diagnoses': 'Atrial fibrillation, Dilated cardiomyopathy', 'Smoker': 'no', 'Number of coronary vessels involved': 'unknown', 'Infarction date (acute)': 'n/a', 'Previous infarction (1) date': 'n/a', 'Previous infarction (2) date': 'n/a', 'Catheterization date': 'n/a', 'Ventriculography': 'n/a', 'Chest X-ray': 'n/a', 'Peripheral blood Pressure (syst/diast)': 'n/a', 'Pulmonary artery pressure (at rest) (syst/diast)': 'n/a', 'Pulmonary artery pressure (at rest) (mean)': 'n/a', 'Pulmonary capillary wedge

In [352]:
#!cp /content/patient.csv '/content/drive/MyDrive/Projeto-Tecnicas'

In [347]:
patient = pd.read_csv('/content/patient.csv')
infarction = pd.read_csv('/content/infarction.csv')
signal = pd.read_csv('/content/signal.csv')
therapy = pd.read_csv('/content/therapy.csv')
hemod = pd.read_csv('/content/hemodinamics.csv')

In [343]:
patient

,idP,age,sex,Smoker,Reason for admission,Additional diagnoses
0,0,62,male,no,Cardiomyopathy,"Atrial fibrillation, Dilated cardiomyopathy"
1,1,39,male,unknown,Cardiomyopathy,"Atrial fibrillation, Hypertropic cardiomyopathy"
2,2,66,male,yes,Myocardial infarction,Arterial hypertension
3,3,57,male,no,Cardiomyopathy,"Atrial fibrillation, Dilated cardiomyopathy"


In [344]:
signal

,idP,idS,fs,sig_len,n_sig,base_date,base_time,units,sig_name,Signal
0,0,0,1000,14900,15,NaN,NaN,"['mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV...","['i', 'ii', 'iii', 'avr', 'avl', 'avf', 'v1', ...",[[ 1.67 -0.2055 -1.875 ... -0.051 0.0085 ...
1,1,0,1000,14900,15,NaN,NaN,"['mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV...","['i', 'ii', 'iii', 'avr', 'avl', 'avf', 'v1', ...",[[-0.1585 0.0495 0.2075 ... 0.121 0.0015 ...
2,1,1,1000,14900,15,NaN,NaN,"['mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV...","['i', 'ii', 'iii', 'avr', 'avl', 'avf', 'v1', ...",[[-0.237 -0.9115 -0.674 ... 0.451 -0.54 ...
3,2,0,1000,14900,15,NaN,NaN,"['mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV...","['i', 'ii', 'iii', 'avr', 'avl', 'avf', 'v1', ...",[[-1.1085 -1.081 0.0275 ... 1.128 -0.713 ...
4,2,1,1000,14900,15,NaN,NaN,"['mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV...","['i', 'ii', 'iii', 'avr', 'avl', 'avf', 'v1', ...",[[-0.5185 0.978 1.496 ... -0.108 -0.4045 ...
5,2,2,1000,14900,15,NaN,NaN,"['mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV...","['i', 'ii', 'iii', 'avr', 'avl', 'avf', 'v1', ...",[[-0.743 -0.672 0.071 ... -0.077 -0.2735 ...
6,3,0,1000,14900,15,NaN,NaN,"['mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV...","['i', 'ii', 'iii', 'avr', 'avl', 'avf', 'v1', ...",[[-0.279 1.228 1.5075 ... 0.492 1.534 ...


In [ ]:
#def __init__(self,dictionary,signal,new_patient)

## Utils

### Baixando a base de dados e gerando um json para guardar os nomes de todos os registros para cada paciente

In [190]:
#!cp '/content/drive/MyDrive/Projeto-Tecnicas/ptb-diagnostic-ecg-database-1.0.0.zip' /content

In [ ]:
#!7z x '/content/ptb-diagnostic-ecg-database-1.0.0.zip'

In [353]:
path_orig = '/content/ptb-diagnostic-ecg-database-1.0.0'
paths = [paths for paths in os.listdir(path_orig) if os.path.isdir(os.path.join(path_orig,paths)) == True]
patient_heas = {}
ids = 0
for i in paths:
  list_hea = [hea for hea in os.listdir(os.path.join(path_orig,i)) if hea.endswith('.hea')]
  patient_heas[i] = {'id': ids, 'heas': list_hea}
  ids +=1

print(patient_heas)

{'patient253': {'id': 0, 'heas': ['s0488_re.hea']}, 'patient201': {'id': 1, 'heas': ['s0420_re.hea', 's0423_re.hea']}, 'patient087': {'id': 2, 'heas': ['s0330lre.hea', 's0321lre.hea', 's0326lre.hea']}, 'patient254': {'id': 3, 'heas': ['s0489_re.hea']}, 'patient065': {'id': 4, 'heas': ['s0282lre.hea', 's0229lre.hea', 's0226lre.hea', 's0221lre.hea']}, 'patient196': {'id': 5, 'heas': ['s0002_re.hea']}, 'patient059': {'id': 6, 'heas': ['s0208lre.hea']}, 'patient283': {'id': 7, 'heas': ['s0542_re.hea']}, 'patient185': {'id': 8, 'heas': ['s0336lre.hea']}, 'patient121': {'id': 9, 'heas': ['s0311lre.hea']}, 'patient277': {'id': 10, 'heas': ['s0527_re.hea']}, 'patient034': {'id': 11, 'heas': ['s0158lre.hea', 's0109lre.hea', 's0118lre.hea', 's0123lre.hea']}, 'patient007': {'id': 12, 'heas': ['s0078lre.hea', 's0038lre.hea', 's0026lre.hea', 's0029lre.hea']}, 'patient017': {'id': 13, 'heas': ['s0075lre.hea', 's0053lre.hea', 's0063lre.hea', 's0055lre.hea']}, 'patient019': {'id': 14, 'heas': ['s0058l

In [358]:
import json 
with open("/content/drive/MyDrive/Projeto-Tecnicas/patient_heas.json", "w") as outfile:
    json.dump(patient_heas, outfile,indent = 4)

### Reading and display

In [ ]:
# Can also read the same files hosted on Physionet s0021bre
signals, fields = wfdb.rdsamp('s0021bre', channels=[*range(15)], sampfrom=0, sampto=5000, pn_dir='ptbdb/patient005/')

In [ ]:
fields

{'fs': 1000,
 'sig_len': 5000,
 'n_sig': 15,
 'base_date': None,
 'base_time': None,
 'units': ['mV',
  'mV',
  'mV',
  'mV',
  'mV',
  'mV',
  'mV',
  'mV',
  'mV',
  'mV',
  'mV',
  'mV',
  'mV',
  'mV',
  'mV'],
 'sig_name': ['i',
  'ii',
  'iii',
  'avr',
  'avl',
  'avf',
  'v1',
  'v2',
  'v3',
  'v4',
  'v5',
  'v6',
  'vx',
  'vy',
  'vz'],
 'comments': ['age: 74',
  'sex: male',
  'ECG date: 24/10/1990',
  'Diagnose:',
  'Reason for admission: Myocardial infarction',
  'Acute infarction (localization): anterior',
  'Former infarction (localization): no',
  'Additional diagnoses: Skoliosis',
  'Smoker: no',
  'Number of coronary vessels involved: 2',
  'Infarction date (acute): 23-Oct-90',
  'Previous infarction (1) date: n/a',
  'Previous infarction (2) date: n/a',
  'Hemodynamics:',
  'Catheterization date: 30-Oct-90',
  'Ventriculography: Akinesia antero-lateral and infero-apical wall',
  'Chest X-ray: normal',
  'Peripheral blood Pressure (syst/diast):  140/70 mmHg',
  'Pul

In [ ]:
fields['comments']

['age: 74',
 'sex: male',
 'ECG date: 24/10/1990',
 'Diagnose:',
 'Reason for admission: Myocardial infarction',
 'Acute infarction (localization): anterior',
 'Former infarction (localization): no',
 'Additional diagnoses: Skoliosis',
 'Smoker: no',
 'Number of coronary vessels involved: 2',
 'Infarction date (acute): 23-Oct-90',
 'Previous infarction (1) date: n/a',
 'Previous infarction (2) date: n/a',
 'Hemodynamics:',
 'Catheterization date: 30-Oct-90',
 'Ventriculography: Akinesia antero-lateral and infero-apical wall',
 'Chest X-ray: normal',
 'Peripheral blood Pressure (syst/diast):  140/70 mmHg',
 'Pulmonary artery pressure (at rest) (syst/diast):   27/14 cmH2O',
 'Pulmonary artery pressure (at rest) (mean): 22 cmH2O',
 'Pulmonary capillary wedge pressure (at rest): 18 cmH2O',
 'Cardiac output (at rest): 4,34 l/min',
 'Cardiac index (at rest): 2,47 l/min/sqrmBSA',
 'Stroke volume index (at rest):  26.5 ml/beat',
 'Pulmonary artery pressure (laod) (syst/diast):   52/32 cmH2O',


In [ ]:
comment = pd.DataFrame(fields['comments'])

,0
0,age: 74
1,sex: male
2,ECG date: 24/10/1990
3,Diagnose:
4,Reason for admission: Myocardial infarction
5,Acute infarction (localization): anterior
6,Former infarction (localization): no
7,Additional diagnoses: Skoliosis
8,Smoker: no
9,Number of coronary vessels involved: 2


In [ ]:
import sqlite3

In [ ]:
conn = sqlite3.connect('test.db')
 
conn.execute('''
CREATE TABLE Record (
   Record VARCHAR PRIMARY KEY NOT NULL,
   Age INTEGER NOT NULL ,
   Sex BOOLEAN NOT NULL 
 );''')
 
conn.commit()
 
print("Records table created");

Records table created


In [ ]:
conn.execute("INSERT INTO Record VALUES (?,?,?)", 's0021bre',fields['comments']['age'],"TRUE")

In [ ]:
conn.close()